### **Exercise: Sentiment Analysis and Key Insights Extraction from Ford Car Reviews**

### **Problem Statement:**
You have been provided with a dataset containing Ford car reviews. Your task is to use LangChain and the concepts you’ve learned to perform the following tasks:

1. **Sentiment Analysis**: Analyze the sentiment of each review, categorize it as positive, neutral, or negative, and store the result.
2. **Key Insights Extraction**: Extract key pieces of information from each review, such as the pros and cons mentioned, and the specific features the reviewer liked or disliked (e.g., vehicle performance, comfort, price).

You will build a LangChain-based solution that leverages language models to automatically extract this information and provide a structured summary of the reviews.

---
### **Steps to Solve:**

#### **Step 1: Load the Dataset**
- The dataset file is named `ford_car_reviews.csv` and is sourced from Kaggle: [Edmunds Consumer Car Ratings and Reviews](https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews).
- For this exercise, **limit the data to the first 25 records**. This can be achieved by using `df.head(25)` or `df.iloc[:25]` when loading the data into a DataFrame.

#### **Step 2: Define the Sentiment Analysis Task**
- Use LangChain to create a pipeline to classify the sentiment of each review.
- Define prompts that can guide the model to evaluate the sentiment. For example:
  - "Given the following car review, classify the sentiment as positive, neutral, or negative."

#### **Step 3: Key Insights Extraction**
- Use LangChain to create a pipeline to extract pros, cons, and notable features from each review. Define prompts such as:
  - "What are the pros and cons of the vehicle described in the following review?"
  - "What specific features of the vehicle does the reviewer like or dislike?"

#### **Step 4: Update the DataFrame with New Information**
- Run the pipeline for each review and collect the sentiment and insights.
- Once the analysis and extraction are complete, update the original DataFrame with additional columns to include:
  - Sentiment (positive, neutral, negative)
  - Pros
  - Cons
  - Liked_Features
  - Disliked_Features

---

### **Example Output:**

```json
{
  "Review_Date": "03/07/13",
  "Vehicle_Title": "2006 Ford Mustang Coupe",
  "Review_Text": "With the expected arrival of our 6th child...",
  "Rating": 4.125,
  "Sentiment": "Positive",
  "Pros": "Good driving experience, Large seating capacity, Great options",
  "Cons": "None mentioned",
  "Liked_Features": ["Driving experience", "Seating capacity", "Options available"],
  "Disliked_Features": []
}
```

In [5]:
%pip install --quiet langchain-community==0.3.0 langgraph==0.2.22 langchain-groq==0.2.0 python-dotenv

In [13]:
import os
import json
import re
from getpass import getpass
from google.colab import userdata
from dotenv import load_dotenv

load_dotenv(".env", override=True)

if "GROQ_API_KEY" not in os.environ:
    try:
        os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    except Exception:
        os.environ["GROQ_API_KEY"] = getpass("Enter your GROQ API Key: ")


In [14]:
# prompt: load csv data from "ford_car_reviews.csv"

import pandas as pd

# Load the CSV data with a limit of 25 records
df = pd.read_csv('ford_car_reviews.csv', nrows=25)

# Now you can work with the first 25 records of the DataFrame 'df'
# Example: print the first 5 rows
print(df.head())

   Unnamed: 0                  Review_Date  Author_Name  \
0           0   on 06/06/18 14:19 PM (PDT)       Vicki    
1           1   on 08/12/17 06:06 AM (PDT)         Tom    
2           2   on 06/15/17 05:43 AM (PDT)         Ray    
3           3   on 05/18/17 17:33 PM (PDT)  Don Watson    
4           4   on 01/03/16 18:03 PM (PST)   One owner    

                                       Vehicle_Title  \
0  2006 Ford Mustang Coupe GT Premium 2dr Coupe (...   
1  2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...   
2  2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...   
3  2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...   
4  2006 Ford Mustang Coupe GT Premium 2dr Coupe (...   

                            Review_Title  \
0                        2006 Mustang GT   
1                              DREAM CAR   
2                             Great Ride   
3  I have wanted a Mustang for 40 years.   
4                              One owner   

                                           

In [15]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import JsonOutputParser

# Initialize the LLM
model_id = "llama3-8b-8192" #llama3-8b-8192, llama-3.1-8b-instant, llama3-groq-8b-8192-tool-use-preview, llama3-groq-70b-8192-tool-use-preview
llm = ChatGroq(model_name=model_id, temperature=0, ) # You need to set your OpenAI API key

# Define the sentiment analysis prompt
sentiment_template = """Given the following car review, classify the sentiment as positive, neutral, or negative. return only the sentiment, i.e. "positive", "neutral", or "negative".

Review: {review}

Sentiment:"""
sentiment_prompt = PromptTemplate(template=sentiment_template, input_variables=["review"])
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt)

# Apply sentiment analysis to each review and update DataFrame
def analyze_sentiment(review):
    sentiment = sentiment_chain.invoke({"review": review})
    return sentiment["text"]

df['Sentiment'] = df['Review'].apply(analyze_sentiment)

# Display the updated DataFrame with sentiment analysis
print(df.head())
# display(df["Sentiment"])

   Unnamed: 0                  Review_Date  Author_Name  \
0           0   on 06/06/18 14:19 PM (PDT)       Vicki    
1           1   on 08/12/17 06:06 AM (PDT)         Tom    
2           2   on 06/15/17 05:43 AM (PDT)         Ray    
3           3   on 05/18/17 17:33 PM (PDT)  Don Watson    
4           4   on 01/03/16 18:03 PM (PST)   One owner    

                                       Vehicle_Title  \
0  2006 Ford Mustang Coupe GT Premium 2dr Coupe (...   
1  2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...   
2  2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...   
3  2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...   
4  2006 Ford Mustang Coupe GT Premium 2dr Coupe (...   

                            Review_Title  \
0                        2006 Mustang GT   
1                              DREAM CAR   
2                             Great Ride   
3  I have wanted a Mustang for 40 years.   
4                              One owner   

                                           

In [16]:
df["Sentiment"].describe()

,Sentiment
count,25
unique,3
top,positive
freq,13


In [18]:
# Define the insights extraction prompt
insights_template = """
Analyze the following car review and extract the pros, cons, and specific features the reviewer liked or disliked. Return the answer as a JSON object with the following keys: pros, cons, liked_features, disliked_features. Do not add anything to the response except JSON

Review: {review}

{{
  "pros": "",
  "cons": "",
  "liked_features": [],
  "disliked_features": []
}}
"""

insights_prompt = PromptTemplate(template=insights_template, input_variables=["review"])
json_parser = JsonOutputParser()
insights_chain = LLMChain(llm=llm, prompt=insights_prompt, output_parser=json_parser)


# Apply insights extraction to each review and update DataFrame
def extract_insights(review):
    insights = insights_chain.invoke({"review": review})
    return insights["text"]

df['Insights'] = df['Review'].apply(extract_insights)

# Process the insights and update the DataFrame
def process_insights(insights):
    return insights['pros'], insights['cons'], insights['liked_features'], insights['disliked_features']


df[['Pros', 'Cons', 'Liked_Features', 'Disliked_Features']] = df['Insights'].apply(lambda x: pd.Series(process_insights(x)))


# Display the updated DataFrame with insights
df.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,Sentiment,Insights,Pros,Cons,Liked_Features,Disliked_Features
0,0,on 06/06/18 14:19 PM (PDT),Vicki,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Mustang GT,Doesn’t disappoint,5.0,positive,"{'pros': 'The car's performance is impressive,...","The car's performance is impressive, with a sm...",The infotainment system can be slow to respond...,"[performance, interior, comfortable seating, p...","[infotainment system, rearview camera view]"
1,1,on 08/12/17 06:06 AM (PDT),Tom,2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...,DREAM CAR,I bought mine 4/17 with 98K. Have been wantin...,3.0,negative,"{'pros': 'Great mileage, Good power, Sounds go...","Great mileage, Good power, Sounds good","Orneriest transmission I've ever used, Harsh r...","[Engine, Appearance]","[Transmission, Ride, Road noise]"
2,2,on 06/15/17 05:43 AM (PDT),Ray,2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...,Great Ride,There will always be a 05-09 mustang for sale...,5.0,positive,"{'pros': 'fairly reasonable', 'cons': '', 'lik...",fairly reasonable,,[great investment],[]
3,3,on 05/18/17 17:33 PM (PDT),Don Watson,2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...,I have wanted a Mustang for 40 years.,I bought my car from an auction I work at ( A...,5.0,positive,"{'pros': 'I love it!!', 'cons': '', 'liked_fea...",I love it!!,,"[v6, air aid cold air injector, throttle body ...",[]
4,4,on 01/03/16 18:03 PM (PST),One owner,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,One owner,I bought this car spankin new and i still am ...,5.0,positive,"{'pros': 'hugs the road, does whatever you ask...","hugs the road, does whatever you ask at a mome...","alternator needed to be fixed, repairs like ti...","[handling, responsiveness]","[alternator issue, need for routine maintenance]"


In [19]:
df

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,Sentiment,Insights,Pros,Cons,Liked_Features,Disliked_Features
0,0,on 06/06/18 14:19 PM (PDT),Vicki,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Mustang GT,Doesn’t disappoint,5.000,positive,"{'pros': 'The car's performance is impressive,...","The car's performance is impressive, with a sm...",The infotainment system can be slow to respond...,"[performance, interior, comfortable seating, p...","[infotainment system, rearview camera view]"
1,1,on 08/12/17 06:06 AM (PDT),Tom,2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...,DREAM CAR,I bought mine 4/17 with 98K. Have been wantin...,3.000,negative,"{'pros': 'Great mileage, Good power, Sounds go...","Great mileage, Good power, Sounds good","Orneriest transmission I've ever used, Harsh r...","[Engine, Appearance]","[Transmission, Ride, Road noise]"
2,2,on 06/15/17 05:43 AM (PDT),Ray,2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...,Great Ride,There will always be a 05-09 mustang for sale...,5.000,positive,"{'pros': 'fairly reasonable', 'cons': '', 'lik...",fairly reasonable,,[great investment],[]
3,3,on 05/18/17 17:33 PM (PDT),Don Watson,2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...,I have wanted a Mustang for 40 years.,I bought my car from an auction I work at ( A...,5.000,positive,"{'pros': 'I love it!!', 'cons': '', 'liked_fea...",I love it!!,,"[v6, air aid cold air injector, throttle body ...",[]
4,4,on 01/03/16 18:03 PM (PST),One owner,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,One owner,I bought this car spankin new and i still am ...,5.000,positive,"{'pros': 'hugs the road, does whatever you ask...","hugs the road, does whatever you ask at a mome...","alternator needed to be fixed, repairs like ti...","[handling, responsiveness]","[alternator issue, need for routine maintenance]"
5,5,on 10/24/15 12:40 PM (PDT),Adam,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,Poor,"Lots of problems with Ford these days, sensor...",3.000,negative,"{'pros': '', 'cons': 'sensors issues, cam phas...",,"sensors issues, cam phasers, and got solenoid ...",[],[]
6,6,on 10/29/11 04:57 AM (PDT),amos247,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,06 mustang gt with a few mods,Bought mine used 20k on it and have added a S...,4.625,positive,"{'pros': 'great bang for the buck performance,...","great bang for the buck performance, decent fu...",,"[SCT tuner, Ford CAI, Flow master mufflers, go...",[]
7,7,on 07/25/11 12:15 PM (PDT),dave3012,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Ford Mustang GT Premium 2dr Coupe (4.6L 8...,I bought my preowned 06 a few weeks back and ...,4.375,positive,"{'pros': 'extreme fun, 360 horses, 3 year 36K ...","extreme fun, 360 horses, 3 year 36K bumper to ...",,"[Airaid air filter system, Flowmaster exhaust,...",[]
8,8,on 07/21/11 11:28 AM (PDT),ronnzy98,2006 Ford Mustang Coupe GT Deluxe 2dr Coupe (4...,Get Rid of it before 100K,I drive 50 miles each way to work and traded ...,3.500,negative,"{'pros': '', 'cons': 'significant loss of powe...",,"significant loss of power, knocking noise, cam...",[],"[cam phaser issue, high repair costs]"
9,9,on 12/06/10 00:00 AM (PST),Anonymous,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,Get this car.,This car is just awesome. The 4.6L V8 makes ...,4.625,positive,"{'pros': 'awesome, fast, better looking, more ...","awesome, fast, better looking, more of a car t...",,"[4.6L V8, stock exhaust, nice rumble, no probl...",[none mentioned]
